# 词向量

基于三层神经网络构建n-gram语言模型(词向量顺带着就算出来了)的基本思路：
![img](https://ws2.sinaimg.cn/large/69d4185bly1fyhuwluj9gj20j40gptbc.jpg)

最下面的w是词，其上面的C(w)是词向量，词向量一层也就是神经网络的输入层(第一层)，这个输入层是一个(n-1)×m的矩阵，其中n-1是词向量数目，m是词向量维度

第二层(隐藏层)是就是普通的神经网络，以H为权重，以tanh为激活函数

第三层(输出层)有|V|个节点，|V|就是词表的大小，输出以U为权重，以softmax作为激活函数以实现归一化，最终就是输出可能是某个词的概率。

**词向量的应用**

第一种应用是找同义词。具体应用案例就是google的word2vec工具，通过训练好的词向量，指定一个词，可以返回和它cos距离最相近的词并排序。

第二种应用是词性标注和语义角色标注任务。具体使用方法是：把词向量作为神经网络的输入层，通过前馈网络和卷积网络完成。

第三种应用是句法分析和情感分析任务。具体使用方法是：把词向量作为递归神经网络的输入。

第四种应用是命名实体识别和短语识别。具体使用方法是：把词向量作为扩展特征使用。

另外词向量有一个非常特别的现象：C(king)-C(queue)≈C(man)-C(woman)，这里的减法就是向量逐维相减，换个表达方式就是：C(king)-C(man)+C(woman)和它最相近的向量就是C(queue)，这里面的原理其实就是：语义空间中的线性关系。基于这个结论相信会有更多奇妙的功能出现。

# word2vec

**表示方法**

问你这样一个问题：如果你大脑有很多记忆单元，让你记住一款白色奥迪Q7运动型轿车，你会用几个记忆单元？你也许会用一个记忆单元，因为这样最节省你的大脑。那么我们再让你记住一款小型灰色雷克萨斯，你会怎么办？显然你会用另外一个记忆单元来记住它。那么如果让你记住所有的车，你要耗费的记忆单元就不再是那么少了，这种表示方法叫做localist representation。这时你可能会换另外一种思路：我们用几个记忆单元来分别识别大小、颜色、品牌等基础信息，这样通过这几个记忆单元的输出，我们就可以表示出所有的车型了。这种表示方法叫做distributed representation，词向量就是一种用distributed representation表示的向量

localist representation中文释义是稀疏表达，典型的案例就是one hot vector，也就是这样的一种向量表示：
```
[1, 0, 0, 0, 0, 0……]表示成年男子
[0, 1, 0, 0, 0, 0……]表示成年女子
[0, 0, 1, 0, 0, 0……]表示老爷爷
[0, 0, 0, 1, 0, 0……]表示老奶奶
[0, 0, 0, 0, 1, 0……]表示男婴
[0, 0, 0, 0, 0, 1……]表示女婴
```

而distributed representation中文释义是分布式表达，上面的表达方式可以改成：
```
性别      老年    成年     婴儿
[1,       0,      1,      0]表示成年男子
[0,       0,      1,      0]表示成年女子
[1,       1,      0,      0]表示老爷爷
[0,       1,      0,      0]表示老奶奶
[1,       0,      0,      1]表示男婴
[0,       0,      0,      1]表示女婴
```

如果我们想表达男童和女童，只需要增加一个特征维度即可.

**word embedding**

翻译成中文叫做词嵌入，这里的embedding来源于范畴论，在范畴论中称为morphism(态射)，态射表示两个数学结构中保持结构的一种过程抽象，比如“函数”、“映射”，他们都是表示一个域和另一个域之间的某种关系。

范畴论中的嵌入(态射)是要保持结构的，而word embedding表示的是一种*“降维”的嵌入*，通过降维避免维度灾难，降低计算复杂度，从而更易于在深度学习中应用。

理解了distributed representation和word embedding的概念，我们就初步了解了word2vec的本质，它其实是通过distributed representation的表达方式来表示词，而且通过降维的word embedding来减少计算量的一种方法。

**CBOW**

CBOW全称是Continuous Bag-of-Words Model，是在已知当前词的上下文的前提下预测当前词：
![img](https://ws2.sinaimg.cn/large/69d4185bly1fyhv70m1ixj20850900sv.jpg)

CBOW模型的神经网络结构设计如下：
- 输入层：词w的上下文一共2c个词的词向量
- 投影层：将输入层的2c个向量做求和累加
- 输出层：一个霍夫曼树，其中叶子节点是语料中出现过的词，权重是出现的次数

与神经网络相比，神经网络模型把首尾相接改成了求和累加，这样减少了维度；去掉了隐藏层，这样减少了计算量；输出层由softmax归一化运算改成了*霍夫曼树*；这一系列修改对训练的性能有很大提升，而效果不减，这是独到之处。

**Skip-gram模型**

Skip-gram全称是Continuous Skip-gram Model，是在已知当前词的情况下预测上下文：
![img](https://ws4.sinaimg.cn/large/69d4185bly1fyhvajgsn2j206g08sdfx.jpg)

Skip-gram模型的神经网络结构设计如下：
- 输入层：w的词向量v(w)
- 投影层：依然是v(w)，就是一个形式
- 输出层：和CBOW一样的霍夫曼树

# RNN

递归神经网络（RNN）是两种人工神经网络的总称。一种是时间递归神经网络（recurrent neural network），另一种是结构递归神经网络（recursive neural network）。时间递归神经网络的神经元间连接构成有向图，而结构递归神经网络利用相似的神经网络结构递归构造更为复杂的深度网络。

传统的神经网络叫做FNN(Feed-Forward Neural Networks)，也就是前向反馈神经网络，RNN是在此基础上引入了定向循环，也就是已神经元为节点组成的图中存在有向的环，这种神经网络可以表达某些前后关联关系，事实上，真正的生物神经元之间也是存在这种环形信息传播的，RNN也是神经网络向真实生物神经网络靠近的一个进步。一个典型的RNN是这样的：
![img](https://ws2.sinaimg.cn/large/69d4185bly1fyhvgad92vj20jl09a0ut.jpg)

图中隐藏层中的节点之间构成了全连接，也就是一个隐藏层节点的输出可以作为另一个隐藏层节点甚至它自己的输入,这种结构可以抽象成：
![image](https://wx1.sinaimg.cn/large/69d4185bly1fyhvhqz0qoj203s082mx6.jpg)

其中U、V、W都是变换概率矩阵，x是输入，o是输出，比较容易看出RNN的关键是隐藏层，因为隐藏层能够捕捉到序列的信息，也就是一种记忆的能力，在RNN中U、V、W的参数都是共享的，也就是只需要关注每一步都在做相同的事情，只是输入不同，这样来降低参数个数和计算量。RNN在NLP中的应用比较多，因为语言模型就是在已知已经出现的词的情况下预测下一个词的概率的，这正是一个有时序的模型，下一个词的出现取决于前几个词，刚好对应着RNN中隐藏层之间的内部连接。

**训练方法**

我们假设x表示输入值，s表示输入x经过U矩阵变换后的值，h表示隐藏层的激活值，o表示输出层的值, f表示隐藏层的激活函数，g表示输出层的激活函数：
- 当t=0时，输入为 $x_0$, 隐藏层为 $h_0$
- 当t=1时，输入为$x_1, s_1 = Ux_1+Wh_0, h_1 = f(s_1), o_1 = g(Vh_1)$
- 当t=2时，$s_2 = Ux_2+Wh_1, h_2 = f(s_2), o_2 = g(Vh_2)$

以此类推，$s_t = Ux_t + Wh_{(t-1)}, ht = f(s_t), o_t = g(Vh_t)$， 如下图：
![img](https://wx3.sinaimg.cn/large/69d4185bly1fyhvti6avuj20bt08ejrw.jpg)

下面就是反向修正参数的过程了，每一步输出o和实际的o值总会有误差，和传统神经网络反向更新的方法一样，用误差来反向推导，利用链式求导求出每层的梯度，从而更新参数，反向推导过程中我们还是把神经网络结构看成展开后的样子：
![img](https://ws3.sinaimg.cn/large/69d4185bly1fyhvu20axyj20bi08h3z1.jpg)

根据链式求导法则，得出隐藏层的残差计算公式为：
$$
\large{
\delta_{t}^{h} = (V^T\delta_{t}^{0} + W^T\delta_{t+1}^{h})*f'(s_t)
}
$$

因此W和U的梯度就是：
$$
\nabla W = \sum \delta_{t}^{h} \times h_t \\
\nabla U = \sum \delta_{t}^{h} \times x_t
$$

**LSTM**

第一张图是传统RNN的另一种形式的示意图，它只包含一个隐藏层，以tanh为激发函数，这里面的“记忆”体现在t的滑动窗口上，也就是有多少个t就有多少记忆，如下图:
![img](https://wx3.sinaimg.cn/large/69d4185bly1fyhw14cx1nj20ja076my3.jpg)

那么我们看LSTM的设计，如下，这里面有一些符号，其中黄色方框是神经网络层(意味着有权重系数和激活函数，σ表示sigmoid激活函数，tanh表示tanh激活函数)，粉红圆圈表示矩阵运算(矩阵乘或矩阵加):
![img](https://wx2.sinaimg.cn/large/69d4185bly1fyhw21j78dj20ji07gjsp.jpg)

这里需要分部分来说，下面这部分是一个历史信息的传递和记忆，其中粉红×是就像一个能调大小的阀门(乘以一个0到1之间的系数)，下面的第一个sigmoid层计算输出0到1之间的系数，作用到粉红×门上，这个操作表达上一阶段传递过来的记忆保留多少，忘掉多少，其中的sigmoid公式如下：
$$
\large{
f_t = \sigma\left(W_f \cdot [h_{t-1},x_t] + b_f\right)
}
$$

下面这部分是由上一层的输出 $h_{t-1}$和本层的输入 $x_{t}$ 得出的新信息，存到记忆中：
![img](https://ws4.sinaimg.cn/large/69d4185bly1fyhw6kncn5j208x05t74j.jpg)

其中包括计算输出值 $\tilde{C}_t$部分的 tanh 神经元和计算比例系数的 sigmoid 神经元（这里面既存在sigmoid又存在 tanh 原因在于 sigmoid 取值范围是 [0,1] 天然作为比例系数，而 tanh 取值范围是 [-1,1] 可以作为一个输出值）。其中$i_{t}$和$\tilde{C}_t$计算公式如下：
$$
\large{
\begin{split}
i_{t} &= \sigma\left(W_i \cdot [h_{t-1},x_t] + b_i\right) \\
\tilde{C}_t &= tanh\left(W_C \cdot [h_{t-1},x_t] + b_C\right) \\
C_t &= f_t * C_{t-1} + i_t * \tilde{C}_t
\end{split}
}
$$

下面部分是隐藏层输出h的计算部分，它考虑了当前拥有的全部信息（上一时序隐藏层的输出、本层的输入x和当前整体的记忆信息），其中本单元状态部分C通过tanh激活并做一个过滤(上一时序输出值和当前输入值通过sigmoid激活后的系数):
![img](https://wx1.sinaimg.cn/large/69d4185bly1fyhwcfroz7j209305yq38.jpg)
$$
\large{
\begin{split}
o_{t} &= \sigma\left(W_o \cdot [h_{t-1},x_t] + b_o\right) \\
h_t &= o_t * tanh(C_t)
\end{split}
}
$$

LSTM非常适合在NLP领域应用，比如一句话出现的词可以认为是不同时序的输入x，而在某一时间t出现词A的概率可以通过LSTM计算，因为词A出现的概率是取决于前面出现过的词的，但取决于前面多少个词是不确定的，这正是LSTM所做的存储着记忆信息C，使得能够得出较接近的概率。

---
RNN就是这样一种神经网络，它让隐藏层自身之间存在有向环，从而更接近生物神经网络，也具有了存储记忆的能力，而LSTM作为RNN中更有实用价值的一种，通过它特殊的结构设计实现了永久记忆留存，更适合于NLP，这也为将深度学习应用到自然语言处理开了先河，有记忆是给聊天机器人赋予智能的前提.

# 自动问答

要把深度学习运用到聊天机器人中，关键在于以下几点：
1. 对几种神经网络结构的选择、组合、优化
2. 因为是有关自然语言处理，所以少不了能让机器识别的词向量
3. 当涉及到相似或匹配关系时要考虑相似度计算，典型的方法是cos距离
4. 如果需求涉及到文本序列的全局信息就用CNN或LSTM
5. 当精度不高时可以加层
6. 当计算量过大时别忘了参数共享和池化

# 聊天语料库